In [1]:
# Random Imports
import os
import sys
import yaml
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

# Regular schema dictates that we put DATAPATH
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ['WANDB_NOTEBOOK_NAME'] = 'calibrate.ipynb'

from ionpy.util import Config

# Setup direcrtories
root = Path("/storage/vbutoi/scratch/ESE")
scratch_root = Path("/storage/vbutoi/scratch/ESE")
code_root = Path("/storage/vbutoi/projects/ESE")

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [2]:
%%yaml default_cfg

data:
  iters_per_epoch: '?' 
  train_splits: cal
  val_splits: val 

dataloader:
  batch_size: '?' # Often, we train with a small num of images total. 
  num_workers: 1
  pin_memory: True 

optim: # Unclear if we should tune this or not.
  _class: torch.optim.Adam
  weight_decay: 0.0 
  lr: 1.0e-4

train:
  epochs: '?' # 10 * 100 = 1000 iterations
  eval_freq: 10 
  pretrained_dir: '?'
  checkpoint: 'max-val-dice_score'
  pretrained_select_metric: 'val-dice_score'

loss_func: 
  _class: '?'
  from_logits: True
  batch_reduction: 'mean' 

<IPython.core.display.Javascript object>

In [3]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.experiment.callbacks.ShowPredictions
  epoch:
    - ese.experiment.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: ece_loss 
        phase: val

<IPython.core.display.Javascript object>

In [4]:
%%yaml calibrator_cfgs 

# For calibration methods.
Finetune:
  _class: null
  
TempScaling:
  _class: ese.experiment.models.calibrators.Temperature_Scaling

ImageBasedTS:
  _class: ese.experiment.models.calibrators.ImageBasedTS
  img_channels: 1
  num_classes: 1

LocalTS:
  _class: ese.experiment.models.calibrators.LocalTS
  img_channels: 1
  num_classes: 1
  filters:
    - 4 
    - 4 
    - 4 

<IPython.core.display.Javascript object>

In [5]:
%%yaml experiment_cfg 

name: "ACDC_CalibratorSet_Experiment1_EXPANDED"
# name: "ACDC_CalibratorSet_BadGT_Exp2"

train: 
    pretrained_dir: 
        - "/storage/vbutoi/scratch/ESE/training/07_09_24_ACDC_PixelCELoss/20240709_120223-2YG5-4666f171cc2ff0127caa727844be2be1"
        - "/storage/vbutoi/scratch/ESE/training/07_09_24_ACDC_SoftDiceLoss/20240709_115941-2WD3-0a311a8965d658cbf9bcf19bfaedda25"

model:
    # - TempScaling
    # - ImageBasedTS
    # - LocalTS
    - Finetune

data:
    _class: "ese.experiment.datasets.ACDC"
    label_threshold: null
    version: 0.1

dataloader:
    batch_size: 8

loss_func:
    _class: 
        - ese.experiment.losses.SoftDiceLoss
        - ese.experiment.losses.PixelCELoss

<IPython.core.display.Javascript object>

In [6]:
from ese.experiment.analysis.analysis_utils.submit_utils import get_ese_calibration_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([callbacks_cfg])

cal_cfgs = get_ese_calibration_configs(
    exp_cfg=experiment_cfg,
    base_cfg=base_cfg,
    calibration_model_cfgs=calibrator_cfgs
    # add_date=False # When we want to add things to an old run folder.
)

In [7]:
len(cal_cfgs)

4

## Running Jobs

In [8]:
from ese.experiment.experiment import run_ese_exp, submit_ese_exps, PostHocExperiment

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [9]:
# ####### Run individual jobs
# run_ese_exp(
#     config=cal_cfgs[0], 
#     experiment_class=PostHocExperiment,
#     run_name='debug',
#     show_examples=True,
#     track_wandb=False,
#     gpu='0',
# )

In [10]:
##### Run Batch Jobs
submit_ese_exps(
    config_list=cal_cfgs,
    experiment_class=PostHocExperiment,
    track_wandb=True,
    available_gpus=['0', '1', '2', '3']
)

Submitted job id: 2194954 on gpu: 0.
Submitted job id: 2195070 on gpu: 1.
Submitted job id: 2195238 on gpu: 2.
Submitted job id: 2195432 on gpu: 3.
